## 1. Import Required Libraries

In [44]:
import pandas as pd
import numpy as np


## 2. Load Raw Dataset

In [45]:
df = pd.read_csv("../Data/Raw/Raw_Data.csv")
df.head()


,timestamp,solving_id,question_id,user_answer,elapsed_time,user_id
0,1565096190868,1,q5012,b,38000,u1
1,1565096221062,2,q4706,c,24000,u1
2,1565096293432,3,q4366,b,68000,u1
3,1565096339668,4,q4829,a,42000,u1
4,1565096401774,5,q6528,b,59000,u1


In [46]:
MAX_TIME = 700

df["elapsed_time"] = df["elapsed_time"].clip(upper=MAX_TIME)


## 3. Inspect Dataset Structure


In [47]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260019 entries, 0 to 260018
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   timestamp     260019 non-null  int64 
 1   solving_id    260019 non-null  int64 
 2   question_id   260019 non-null  object
 3   user_answer   260008 non-null  object
 4   elapsed_time  260019 non-null  int64 
 5   user_id       260019 non-null  object
dtypes: int64(3), object(3)
memory usage: 11.9+ MB


## 4. Convert Timestamp to Datetime

The `timestamp` column is stored as Unix time in milliseconds and is converted
to a UTC datetime format.

In [48]:
df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms", utc=True)


In [49]:
df.head()


,timestamp,solving_id,question_id,user_answer,elapsed_time,user_id
0,2019-08-06 12:56:30.868000+00:00,1,q5012,b,700,u1
1,2019-08-06 12:57:01.062000+00:00,2,q4706,c,700,u1
2,2019-08-06 12:58:13.432000+00:00,3,q4366,b,700,u1
3,2019-08-06 12:58:59.668000+00:00,4,q4829,a,700,u1
4,2019-08-06 13:00:01.774000+00:00,5,q6528,b,700,u1


## 5. Convert Elapsed Time to Seconds

Elapsed time is converted from milliseconds to seconds to improve interpretability.

In [50]:
df["elapsed_time_seconds"] = df["elapsed_time"] / 1000


In [51]:
df.head()

,timestamp,solving_id,question_id,user_answer,elapsed_time,user_id,elapsed_time_seconds
0,2019-08-06 12:56:30.868000+00:00,1,q5012,b,700,u1,0.7
1,2019-08-06 12:57:01.062000+00:00,2,q4706,c,700,u1,0.7
2,2019-08-06 12:58:13.432000+00:00,3,q4366,b,700,u1,0.7
3,2019-08-06 12:58:59.668000+00:00,4,q4829,a,700,u1,0.7
4,2019-08-06 13:00:01.774000+00:00,5,q6528,b,700,u1,0.7


## 6. Check for Missing Values

We identify missing values to determine appropriate handling strategies.


In [52]:
df.isna().sum()


timestamp                0
solving_id               0
question_id              0
user_answer             11
elapsed_time             0
user_id                  0
elapsed_time_seconds     0
dtype: int64

## 7. Handle Missing User Answers

Rows with missing `user_answer` values are removed because unanswered questions
do not provide valid analytical value.

In [53]:
df = df.dropna(subset=["user_answer"])


In [54]:
df.isna().sum()


timestamp               0
solving_id              0
question_id             0
user_answer             0
elapsed_time            0
user_id                 0
elapsed_time_seconds    0
dtype: int64

In [55]:
df["user_answer"].value_counts()


user_answer
b    74194
a    70325
c    69027
d    46462
Name: count, dtype: int64

## 8. Remove Duplicate Records

Duplicate rows are removed based on user ID, question ID, and timestamp.

In [56]:
df.duplicated(
    subset=["user_id", "question_id", "timestamp"]
).sum()


np.int64(2)

In [57]:
df = df.drop_duplicates(
    subset=["user_id", "question_id", "timestamp"]
)


## 9. Standardize User Answers

User answers are standardized to lowercase and stripped of whitespace
to ensure consistent categorical values.


In [58]:
df["user_answer"] = (
    df["user_answer"]
    .str.lower()
    .str.strip()
)


## 10. Validate Identifiers

We validate key identifiers to ensure correct formatting and data integrity.


In [59]:
# question_id format
df = df[df["question_id"].str.match(r"^q\d+$")]

# user_id validity
df = df[df["user_id"].notna() & (df["user_id"].str.strip() != "")]


## 11. Drop Redundant Columns

The original elapsed time column is removed after conversion to seconds.


In [60]:
df = df.drop(columns=["elapsed_time"])


## 12. Final Data Validation

We perform a final inspection to ensure the dataset is clean and consistent.


In [61]:
df.info()
df.head()


<class 'pandas.core.frame.DataFrame'>
Index: 260006 entries, 0 to 260018
Data columns (total 6 columns):
 #   Column                Non-Null Count   Dtype              
---  ------                --------------   -----              
 0   timestamp             260006 non-null  datetime64[ns, UTC]
 1   solving_id            260006 non-null  int64              
 2   question_id           260006 non-null  object             
 3   user_answer           260006 non-null  object             
 4   user_id               260006 non-null  object             
 5   elapsed_time_seconds  260006 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(1), int64(1), object(3)
memory usage: 13.9+ MB


,timestamp,solving_id,question_id,user_answer,user_id,elapsed_time_seconds
0,2019-08-06 12:56:30.868000+00:00,1,q5012,b,u1,0.7
1,2019-08-06 12:57:01.062000+00:00,2,q4706,c,u1,0.7
2,2019-08-06 12:58:13.432000+00:00,3,q4366,b,u1,0.7
3,2019-08-06 12:58:59.668000+00:00,4,q4829,a,u1,0.7
4,2019-08-06 13:00:01.774000+00:00,5,q6528,b,u1,0.7


## 13. Save Clean Dataset

The cleaned dataset is saved for use in downstream analysis and modeling.


In [62]:
df.to_csv("../Data/Cleaned/cleaned_data.csv", index=False)


In [ ]:
df["elapsed_time_capped"].describe()


count    260006.000000
mean         26.024575
std          24.902371
min           0.000000
25%          15.333000
50%          20.000000
75%          29.250000
max         700.000000
Name: elapsed_time_capped, dtype: float64